In [11]:
import pandas as pd
import pvlib

In [41]:
def calc_power_output(latitude, longitude, panel_area, panel_efficiency, panel_tilt, panel_azimuth, times):
    tz = 'Europe/Dublin'
    
    location = pvlib.location.Location(latitude, longitude)
    #clearsky_irradiance =  location.get_clearsky(times)

    data, months, inputs, meta = pvlib.iotools.get_pvgis_tmy(latitude=latitude, longitude=longitude)
    if data.index.tzinfo is None:
        raise ValueError("data.index must be timezone aware")
    solar_position = location.get_solarposition(data.index)

    #solar_position = location.get_solarposition(date_time)
    # Optimisic estimate of solar irradiation because it assumes clear skies
    # clear_sky = location.get_clearsky(date_time, model="ineichen")
    irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt=panel_tilt,
        surface_azimuth=panel_azimuth,
        solar_zenith=solar_position['zenith'],
        solar_azimuth=solar_position['azimuth'],
        ghi=data['ghi'],
        dni=data['dni'],
        dhi=data['dhi'])
    
    #default values entered for wind_speed, u0, and u1
    module_temperature = pvlib.temperature.faiman(poa_global=irradiance["poa_global"], temp_air=data["temp_air"], wind_speed=1, u0=25.0, u1=6.84)

    dc_power = pvlib.pvsystem.pvwatts_dc(
    g_poa_effective=irradiance['poa_global'],
    # simplified assumptions that module and cell temperature are the same
    temp_cell=module_temperature,
    pdc0=300,
    gamma_pdc=-0.004,
    temp_ref=25.0,
    )


    return dc_power

In [42]:
times = pd.date_range(start='2025-03-29 00:00:00', end='2025-03-30 00:00:00', freq='30min', tz='Europe/Dublin')
clearsky_irradiance = calc_power_output(55.6846, 12.5389, 6, 0.18, 40, 180, times)
clearsky_irradiance


time(UTC)
2016-01-01 00:00:00+00:00    0.0
2016-01-01 01:00:00+00:00    0.0
2016-01-01 02:00:00+00:00    0.0
2016-01-01 03:00:00+00:00    0.0
2016-01-01 04:00:00+00:00    0.0
                            ... 
2016-12-31 19:00:00+00:00    0.0
2016-12-31 20:00:00+00:00    0.0
2016-12-31 21:00:00+00:00    0.0
2016-12-31 22:00:00+00:00    0.0
2016-12-31 23:00:00+00:00    0.0
Length: 8760, dtype: float64